In [ ]:
import pandas as pd
from preprocessor import text_cleaner, selector, p_selector
from connector import connection

Establishing connection with the database.

In [ ]:
con = connection.Connection()
content = con.select_table('text', 'old_text', 'WHERE old_id > 1675 AND old_id < 1699')
con.terminate_connection()

In order to decode our byte data types into strings, it is here necessary to unpack
the elements out of tuples_list and assign them to input_text.
First, we're initializing a new list and will leave it empty:

In [ ]:
input_text = []
for tuples in content:
    input_tuple = tuples
    for bytes_elements in tuples:
        decoded = bytes_elements.decode()
        input_text.append(decoded)

Iterating the preprocessor functions over input_text and assigning the results to a new list.
Parameters could be turned on/off e.g. text_cleaner.clean(x, newline=False)

In [ ]:
text_transformed = list(map(lambda x: text_cleaner.clean(x), input_text))
article_title = list(map(lambda y: selector.find_title(y), text_transformed))
article_subheaders = list(map(lambda z: selector.find_subheader(z), text_transformed))
cleaned_subheaders = selector.clean_subheaders(article_subheaders)

Defining the dictionary that holds our two lists and that will form our new database with the preprocessed data

In [ ]:
data = {
    'title': article_title,
    'article': text_transformed,
    'subheaders': cleaned_subheaders
}

Creating a pandas DataFrame to hold our preprocessed data

In [ ]:
pandas = pd.DataFrame.from_dict(data)
pandas['paragraphs'] = pandas.apply(
    lambda x: p_selector.paragrapher(x.subheaders, x.article), axis=1)
print(pandas.head)

Context manager for parsing the resulting text into a text file.

In [ ]:
with open('output.txt', 'w') as opened_file:
    final = text_transformed[16].strip()
    opened_file.write("titel:" + article_title + "\n")
    opened_file.write(final)